In [1]:
!ls

main.ipynb          main_original.ipynb


In [2]:
%cd ..

/Users/matthew/Documents/options-pricing-monte-carlo


## Performance Benchmark: Optimised Monte Carlo Engine

This section evaluates the performance and accuracy of our Numba-accelerated OptimizedMCEngine. We measure its speed, memory efficiency, and accuracy against the analytical Black-Scholes price across an increasing number of simulations.

The table below presents several key metrics:

`sims_per_second`: Measures the raw computational throughput of the engine. A higher number is better.

`accuracy_pct`: Shows how close our Monte Carlo price is to the analytical Black-Scholes "ground truth." This validates the correctness of our implementation.

`memory_mb`: Tracks the additional memory allocated during the simulation run, demonstrating the engine's efficiency.

In [4]:
import numpy as np
import pandas as pd
from src.models.monte_carlo import OptimizedMCEngine

# 1. Define the parameters for our test option
S0 = 100      # Initial stock price
K = 105       # Strike price
T = 1.0       # Time to maturity (in years)
r = 0.05      # Risk-free interest rate
sigma = 0.2   # Volatility

# 2. Initialize the high-performance Numba engine
# The first run will be a bit slower as Numba compiles the functions.
print("Initializing and warming up the Numba JIT compiler...")
engine_numba = OptimizedMCEngine(use_numba=True, use_antithetic=True)
print("Initialization complete.")


# 3. Run the benchmark on the high-performance Numba engine
print("\n" + "="*50)
print("PERFORMANCE: Numba-Optimized Engine")
print("="*50)
numba_results = engine_numba.benchmark_performance(S0, K, T, r, sigma)
display(numba_results)

Initializing and warming up the Numba JIT compiler...
Initialization complete.

PERFORMANCE: Numba-Optimized Engine
--- Benchmarking Engine (Numba: True, Antithetic: True) ---


,simulations,time_seconds,sims_per_second,memory_mb,accuracy_pct,mc_price,analytical_price
0,"10,000",0.0025,"4,025,630",0.38,99.0606%,8.09670,8.02135
1,"50,000",0.0014,"37,012,919",0.25,99.2412%,8.08222,8.02135
2,"100,000",0.0007,"152,686,712",0.00,99.2146%,8.08435,8.02135
3,"500,000",0.0043,"116,631,555",0.27,99.9673%,8.01873,8.02135
4,"1,000,000",0.0060,"167,524,224",0.00,99.8485%,8.03351,8.02135


In [5]:
import pandas as pd
from src.models.model_validation import ModelBreakdownAnalyzer

# To display DataFrames nicely
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', '{:.2f}'.format)


# 1. Initialize the analyzer
analyzer = ModelBreakdownAnalyzer(ticker='SPY')

# 2. Run the analysis steps
analyzer.fetch_market_data()
breakdown_df = analyzer.calculate_model_errors()

# 3. Display the detailed breakdown DataFrame
print("\n--- Black-Scholes Model Breakdown Analysis ---")
display(breakdown_df)

# 4. Generate and print the final killer metrics for your CV
print("\n--- Key Performance Metrics ---")
key_metrics = analyzer.generate_key_metrics()
for key, value in key_metrics.items():
    print(f"{key}: {value}")

Fetching option surface and market price for SPY...
Fetched 291 liquid call options.
Calculated breakdown metrics for 291 options.

--- Black-Scholes Model Breakdown Analysis ---


,strike,moneyness,days_to_expiry,market_price,bs_price_at_20_vol,pricing_error_pct,implied_vol_pct,bid_ask_spread_pct
0,265.00,2.41,14,287.00,374.98,30.65,NaN,0.74
1,300.00,2.13,14,328.49,340.04,3.52,NaN,0.00
2,320.00,2.00,14,278.83,320.08,14.80,NaN,0.73
3,340.00,1.88,14,292.30,300.12,2.68,NaN,0.00
4,350.00,1.83,14,282.22,290.14,2.81,NaN,0.00
...,...,...,...,...,...,...,...,...
286,775.00,0.83,112,0.05,1.81,3522.78,11.86,0.00
287,780.00,0.82,112,0.05,1.58,3065.82,12.25,0.00
288,790.00,0.81,112,0.04,1.20,2905.91,12.72,0.00
289,795.00,0.80,112,0.03,1.05,3383.93,12.73,0.00



--- Key Performance Metrics ---
Options Analyzed: 143
Avg Pricing Error (vs 20% Vol): 847.33%
Avg Error for OTM Options: 1373.15%
Avg Error for ITM Options: 21.79%
Implied Vol Range: 8.2% - 36.8%
Avg Bid-Ask Spread: 1.75%


In [6]:
from src.strategies.arbitrage import ArbitrageScanner

# Initialize and run the scanner
scanner = ArbitrageScanner()
violations_df = scanner.scan_put_call_parity_violations()

# Display the results
print("\n--- Put-Call Parity Arbitrage Opportunities ---")
if violations_df.empty:
    print("No significant arbitrage opportunities found.")
else:
    # Display key columns
    display(violations_df[['ticker', 'strike', 'lastPrice_call', 'lastPrice_put', 'violation_amt', 'violation_pct']])

Scanning for Put-Call Parity violations using r=4.23%...

--- Put-Call Parity Arbitrage Opportunities ---


,ticker,strike,lastPrice_call,lastPrice_put,violation_amt,violation_pct
0,SPY,385.00,149.61,0.01,105.05,70.21
1,SPY,480.00,155.49,0.01,4.15,2.67
2,SPY,550.00,86.56,0.01,3.08,3.55
3,SPY,560.00,72.75,0.01,6.89,9.46
4,SPY,565.00,68.99,0.02,5.65,8.20
...,...,...,...,...,...,...
170,TSLA,360.00,0.05,44.06,2.72,5449.99
171,TSLA,367.50,0.03,52.00,3.18,10616.65
172,TSLA,370.00,0.03,54.30,2.98,9949.99
173,TSLA,372.50,0.04,50.60,3.23,8062.51


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.risk.hedging import simulate_delta_hedging

# --- Main Analysis Block for Delta Hedging Simulation ---

# 1. Define the parameters for the option we are hedging
S0 = 100      # Initial stock price
K = 105       # Strike price
T = 1.0       # Time to maturity (in years)
r = 0.05      # Risk-free interest rate
sigma = 0.2   # Assumed volatility for our hedging model
n_simulations = 20000 # Number of market scenarios to simulate
steps = 252   # Daily hedging

# 2. Run the simulation
print("Running Delta Hedging simulation...")
final_pnl = simulate_delta_hedging(S0, K, T, r, sigma, n_simulations, steps)
print("Simulation complete.")

# 3. Analyze and plot the results
mean_pnl = np.mean(final_pnl)
std_pnl = np.std(final_pnl)

print(f"\n--- Hedging Performance Results ---")
print(f"Average Hedging P&L: {mean_pnl:.4f}")
print(f"Standard Deviation of P&L (Hedging Error): {std_pnl:.4f}")

# Plot the distribution of the final P&L
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(12, 7))
sns.histplot(final_pnl, bins=70, kde=True, stat="density")
plt.axvline(mean_pnl, color='r', linestyle='--', label=f'Mean P&L: {mean_pnl:.2f}')
plt.title('Distribution of Final P&L from Delta Hedging', fontsize=16)
plt.xlabel('Final Profit & Loss (P&L)', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.legend()
plt.show()

ImportError: cannot import name 'simulate_gbm_paths' from 'src.models.monte_carlo' (/Users/matthew/Documents/options-pricing-monte-carlo/src/models/monte_carlo.py)